In [5]:
# The idea behind this is that players in the madden video games have 0-99 overall ratings
# and a bunch of other 0-99 ratings for specific attributes like speed, awareness, throw power, etc.
# my idea behind this project is to use regression to get the overall rating from these more specific ratings

In [6]:
import pandas
import statsmodels.api as sm
import numpy as np


In [7]:
# Toy example with quarterbacks

In [8]:
data=pandas.read_csv('Madden_23_Player_Ratings.csv') # Read data
qbs=data[data['Position']=='QB'] # filtering data for quarterbacks 
qb_ratings=qbs[['awareness_rating','throwPower_rating','kickReturn_rating','leadBlock_rating','strength_rating','bCVision_rating','catchInTraffic_rating','playAction_rating','pursuit_rating','mediumRouteRunning_rating','catching_rating','acceleration_rating','spinMove_rating','finesseMoves_rating','spectacularCatch_rating','runBlock_rating','tackle_rating','injury_rating','zoneCoverage_rating','deepRouteRunning_rating','trucking_rating','throwAccuracyShort_rating','jukeMove_rating','playRecognition_rating','shortRouteRunning_rating','breakSack_rating','speed_rating','runBlockPower_rating','jumping_rating','toughness_rating','throwOnTheRun_rating','manCoverage_rating','stiffArm_rating','powerMoves_rating','release_rating','hitPower_rating','throwAccuracyMid_rating','kickAccuracy_rating','passBlockPower_rating','impactBlocking_rating','stamina_rating','carrying_rating','breakTackle_rating','kickPower_rating','throwUnderPressure_rating','passBlock_rating','changeOfDirection_rating','press_rating','throwAccuracyDeep_rating','blockShedding_rating','runBlockFinesse_rating','agility_rating','overall_rating','passBlockFinesse_rating']]
# ^ These ratings are the ones that could matter so I'm only looking at these ones
training=qb_ratings.sample(frac=.8,random_state=1) # take 80% as training data
validation_rows=np.setdiff1d(qb_ratings.index,training.index)
array=np.zeros(data.index.size)
for n in range(data.index.size):
    array[n]=(n in validation_rows)
array=np.array(array,dtype=bool)

validation=data.loc[array]
#print(validation)
validation=validation[['overall_rating','awareness_rating','throwPower_rating','kickReturn_rating','leadBlock_rating','strength_rating','bCVision_rating','catchInTraffic_rating','playAction_rating','pursuit_rating','mediumRouteRunning_rating','catching_rating','acceleration_rating','spinMove_rating','finesseMoves_rating','spectacularCatch_rating','runBlock_rating','tackle_rating','injury_rating','zoneCoverage_rating','deepRouteRunning_rating','trucking_rating','throwAccuracyShort_rating','jukeMove_rating','playRecognition_rating','shortRouteRunning_rating','breakSack_rating','speed_rating','runBlockPower_rating','jumping_rating','toughness_rating','throwOnTheRun_rating','manCoverage_rating','stiffArm_rating','powerMoves_rating','release_rating','hitPower_rating','throwAccuracyMid_rating','kickAccuracy_rating','passBlockPower_rating','impactBlocking_rating','stamina_rating','carrying_rating','breakTackle_rating','kickPower_rating','throwUnderPressure_rating','passBlock_rating','changeOfDirection_rating','press_rating','throwAccuracyDeep_rating','blockShedding_rating','runBlockFinesse_rating','agility_rating','passBlockFinesse_rating']]
# makes the rest of the data the validation set

In [9]:
from patsy import dmatrices

In [10]:
y, X =dmatrices('overall_rating ~ awareness_rating+throwPower_rating+kickReturn_rating+leadBlock_rating+strength_rating+bCVision_rating+catchInTraffic_rating+playAction_rating+pursuit_rating+mediumRouteRunning_rating+catching_rating+acceleration_rating+spinMove_rating+finesseMoves_rating+spectacularCatch_rating+runBlock_rating+tackle_rating+injury_rating+zoneCoverage_rating+deepRouteRunning_rating+trucking_rating+throwAccuracyShort_rating+jukeMove_rating+playRecognition_rating+shortRouteRunning_rating+breakSack_rating+speed_rating+runBlockPower_rating+jumping_rating+toughness_rating+throwOnTheRun_rating+manCoverage_rating+stiffArm_rating+powerMoves_rating+release_rating+hitPower_rating+throwAccuracyMid_rating+kickAccuracy_rating+passBlockPower_rating+impactBlocking_rating+stamina_rating+carrying_rating+breakTackle_rating+kickPower_rating+throwUnderPressure_rating+passBlock_rating+changeOfDirection_rating+press_rating+throwAccuracyDeep_rating+blockShedding_rating+runBlockFinesse_rating+agility_rating+passBlockFinesse_rating',data=training, return_type='dataframe')
# this line does the regression part

In [11]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.params) # see the weight values here

Intercept                   -109.484450
awareness_rating               0.290388
throwPower_rating              0.670634
kickReturn_rating              0.058926
leadBlock_rating               0.034343
strength_rating                0.043156
bCVision_rating               -0.003172
catchInTraffic_rating          0.028398
playAction_rating              0.173222
pursuit_rating                 0.080750
mediumRouteRunning_rating      0.031016
catching_rating                0.021982
acceleration_rating           -0.036984
spinMove_rating                0.030574
finesseMoves_rating            0.233155
spectacularCatch_rating       -0.053904
runBlock_rating               -0.036741
tackle_rating                 -0.007912
injury_rating                  0.097062
zoneCoverage_rating            0.114926
deepRouteRunning_rating       -0.063762
trucking_rating                0.020021
throwAccuracyShort_rating      0.089708
jukeMove_rating                0.067493
playRecognition_rating        -0.004212


In [12]:
significance_arr=res.params.to_numpy()>0
significance_arr[0]=True
new_params=np.multiply(significance_arr,res.params.to_numpy())
#print(np.matmul(validation.to_numpy(),new_params[1:]))
#print(new_params[1:])
#print(qb_ratings.iloc[1])
no_overall=validation[['awareness_rating','throwPower_rating','kickReturn_rating','leadBlock_rating','strength_rating','bCVision_rating','catchInTraffic_rating','playAction_rating','pursuit_rating','mediumRouteRunning_rating','catching_rating','acceleration_rating','spinMove_rating','finesseMoves_rating','spectacularCatch_rating','runBlock_rating','tackle_rating','injury_rating','zoneCoverage_rating','deepRouteRunning_rating','trucking_rating','throwAccuracyShort_rating','jukeMove_rating','playRecognition_rating','shortRouteRunning_rating','breakSack_rating','speed_rating','runBlockPower_rating','jumping_rating','toughness_rating','throwOnTheRun_rating','manCoverage_rating','stiffArm_rating','powerMoves_rating','release_rating','hitPower_rating','throwAccuracyMid_rating','kickAccuracy_rating','passBlockPower_rating','impactBlocking_rating','stamina_rating','carrying_rating','breakTackle_rating','kickPower_rating','throwUnderPressure_rating','passBlock_rating','changeOfDirection_rating','press_rating','throwAccuracyDeep_rating','blockShedding_rating','runBlockFinesse_rating','agility_rating','passBlockFinesse_rating']]
#print(no_overall)
predictions=np.matmul(no_overall.to_numpy(),new_params[1:])+new_params[0]

#print(new_params[1:])

In [13]:
actuals=validation.to_numpy()[:,0]
print(predictions-actuals)

[20.94508025 29.2367633  26.57088913 23.42808284 19.47636933 20.92296861
 20.23047113 22.09989234 28.00636505 21.91416104 35.66459962 17.83772927
 26.23320521 24.9580279  24.3461672  28.61346701 21.3395433  22.63000994
 27.87675008 23.49074851 20.03195915]


In [14]:
#https://www.statsmodels.org/stable/gettingstarted.html
# Website for doing regression with python